# PEC2

![lego](https://kaggle2.blob.core.windows.net/datasets-images/1599/2846/759e9518355eabd33bd70f907e4f7992/dataset-cover.jpg)


Vamos a trabajar sobre un dataset de LEGO, en el que tenemos diferentes ficheros/tablas.
Éste es un esquema de las tablas que representan los CSV de datos que tenemos:

![diagram](downloads_schema.png)

## 1 Carga de Datos
En la tabla `sets` tenemos los diferentes sets de lego, cada uno con su 
- identificador, 
- año 
- y su "temática".


En la tabla `themes` tenemos las diferentes temáticas de todos los sets de lego.

Carga ambos DataFrames, y modifica el esquema si lo crees oportuno.

In [1]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import lower

In [2]:
spark = pyspark.sql.SparkSession.builder.appName('lego').getOrCreate()

In [3]:
# column heads: set_num, name, year, theme_id, num_parts
sets = spark.read.option('header', True).\
    csv('data/sets.csv').\
    toDF('set_num', 'set_name', 'year', 'theme_id', 'num_parts')
sets.show(5)

+-------+--------------------+----+--------+---------+
|set_num|            set_name|year|theme_id|num_parts|
+-------+--------------------+----+--------+---------+
|   00-1|     Weetabix Castle|1970|     414|      471|
| 0011-2|   Town Mini-Figures|1978|      84|       12|
| 0011-3|Castle 2 for 1 Bo...|1987|     199|        2|
| 0012-1|  Space Mini-Figures|1979|     143|       12|
| 0013-1|  Space Mini-Figures|1979|     143|       12|
+-------+--------------------+----+--------+---------+
only showing top 5 rows



In [4]:
#column heads: id, name, parent_id
themes = spark.read.option('header', True).\
    csv('data/themes.csv').\
    toDF('theme_id', 'theme_name', 'parent_id')
themes.show(5)

+--------+--------------+---------+
|theme_id|    theme_name|parent_id|
+--------+--------------+---------+
|       1|       Technic|     null|
|       2|Arctic Technic|        1|
|       3|   Competition|        1|
|       4|Expert Builder|        1|
|       5|         Model|        1|
+--------+--------------+---------+
only showing top 5 rows



## 2 Joins
Ahora queremos extraer **sólo** las siguientes columnas:
 - el identificador del set (`set_num`) - sets.set_num
 - la temática del set (columna `name` de `themes`) - themes.name
 - el nombre del set (columna `name` de `sets`) - sets.name
 
De todos los sets que contengan "x-wing" en su nombre, y sean posteriores a 2011.


Ten en cuenta que "x-wing" puede aparecer en cualquier posición del nombre del set, y con cualquier composición de mayúsculas o minúsculas. (es decir, tienes que detectar apariciones de `x-wing`, `X-Wing`, `X-wing`...etc).

Ten en cuenta que sólo queremos esas 3 columnas en el resultado.

In [5]:
# SETS: 'set_num', 'set_name', 'set_year', 'theme_id', 'set_num_parts'
# THEMES: 'theme_id', 'theme_name', 'theme_parent_id'


# Se que se puede hacer esto en menos etapas, pero creo que de esta forma es mas claro para ti de evaluar:
joinExpression = sets["theme_id"] == themes['theme_id']
both = sets.join(themes, joinExpression)
post_2011 = both.filter(sets["year"] > 2011)
x_wing = post_2011.filter(lower(col("set_name")).rlike("x-wing"))
themes_and_sets = x_wing.select('set_num', 'set_name','theme_name')
themes_and_sets.show()




+-------------+--------------------+--------------------+
|      set_num|            set_name|          theme_name|
+-------------+--------------------+--------------------+
|      10240-1|Red Five X-Wing S...|Star Wars Episode...|
|      30278-1|Poe's X-wing Fighter|                Mini|
|      75032-1|      X-Wing Fighter|                Mini|
|      75102-1|Poe’s X-Wing Figh...|           Star Wars|
|      75125-1|Resistance X-Wing...| Star Wars Episode 7|
|      75149-1|Resistance X-Wing...| Star Wars Episode 7|
|       9493-1|  X-wing Starfighter|Star Wars Episode...|
|       9677-1|X-wing Starfighte...|     Planet Series 2|
|   SWCOMIC1-1|              X-Wing|                Mini|
|   TRUXWING-1|              X-Wing|     Star Wars Other|
|   TRUXWING-2|Poe's X-Wing Fighter|           Star Wars|
|TRUXWINGTIE-1|TRU X-Wing Fighte...|     Star Wars Other|
+-------------+--------------------+--------------------+



## 3 Más joins


- **todos los sets.theme_id que sean 158**  de la temática Star wars
    - Table sets  |  column theme_id, value 158

el resultado debe incluír sólo las siguientes columnas:
 - `set_num`    
     - Table sets | column set_num   


 - `name` 
     - Table sets | column set_name


 - `inventory_id`        
     - Table inventory_sets | columns id/version/set_num  | value id=158
     - cuando no exista informacion, = null


- `quantity`    
     - Table inventory_sets  | column quantity
     - cuando no exista informacion, = null
 


In [6]:
# inventory_sets columns: inventory_id, set_num, quantity
inventory_sets = spark.read.option("header", True).\
    csv("data/inventory_sets.csv").\
    toDF("inventory_id", "set_num", "quantity")
inventory_sets.show(1)

# inventory columns: id, version, set_num
inventories = spark.read.option("header", "true").\
            csv("data/inventories.csv").\
            toDF("id", "version", "set_num")
inventories.show(1)

+------------+-------+--------+
|inventory_id|set_num|quantity|
+------------+-------+--------+
|          35|75911-1|       1|
+------------+-------+--------+
only showing top 1 row

+---+-------+-------+
| id|version|set_num|
+---+-------+-------+
|  1|      1| 7922-1|
+---+-------+-------+
only showing top 1 row



In [7]:
joinOnColumn = sets["set_num"] == inventory_sets['set_num']
joinSetInventory = sets.join(inventory_sets, joinOnColumn)
id_158 = joinSetInventory.filter(sets["theme_id"] == 158)
StarWarsInventory = id_158.select(sets["set_num"], sets['set_name'],inventory_sets['inventory_id'],inventory_sets['quantity']).show()

+-------+--------------------+------------+--------+
|set_num|            set_name|inventory_id|quantity|
+-------+--------------------+------------+--------+
| 4481-1|      Hailfire Droid|       11344|       1|
|75029-1|                 AAT|       12222|       1|
|75035-1|   Kashyyyk Troopers|       12813|       1|
|75037-1| Battle on Saleucami|        4941|       1|
|75038-1|    Jedi Interceptor|        4941|       1|
|75043-1|               AT-AP|       12813|       1|
|75045-1|Republic AV-7 Ant...|        4941|       1|
| 8008-1|       Stormtrooper™|        6181|       1|
| 8010-1|        Darth Vader™|        6181|       1|
| 9497-1|Republic Striker ...|        3110|       1|
+-------+--------------------+------------+--------+

